In [1]:
import cv2
import re
import pandas as pd
import numpy as np
import easyocr
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
train_df

,img_path,text
0,./train/train_00001.png,골목미용실
1,./train/train_00002.png,한성부동산
2,./train/train_00003.png,홍라운지
3,./train/train_00004.png,모단걸응접실
4,./train/train_00005.png,씨앗양식
...,...,...
12154,./train/train_12161.png,넬슨
12155,./train/train_12162.png,앤드루 램버트 지음
12156,./train/train_12163.png,박아람 옮김
12157,./train/train_12164.png,판결의


In [4]:
test_df

,img_path
0,./test/test_00001.png
1,./test/test_00002.png
2,./test/test_00003.png
3,./test/test_00004.png
4,./test/test_00005.png
...,...
3664,./test/test_03669.png
3665,./test/test_03670.png
3666,./test/test_03671.png
3667,./test/test_03672.png


In [9]:
from tensorflow.keras import layers

In [10]:
FIRST_EPOCHS = 10
SECOND_EPOCHS = 10

In [16]:
# Pretrained Model 로드
model = EasyOCR()
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

AttributeError: 'EasyOCR' object has no attribute 'compile'

In [ ]:
easyocr_model = EasyOCR()

In [11]:
def get_accuracy(answer_df, predict_df):
    return accuracy_score(answer_df['text'].values, predict_df['text'].values)

In [12]:
class EasyOCR:
    def __init__(self, lang='kor'):
        self.lang = lang
    
    def load_image(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    def text_preprocessing(self, text):
        result = ''
        for word in text:
            word = word.replace('\n', '')
            word = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', word)
            word = word.strip()
            result += word
        return result
    
    def prediction(self, img_path_list):
        preds = []
        for img_path in tqdm(img_path_list):
            img = self.load_image(img_path)
            reader = easyocr.Reader(['ko', 'en'], gpu = False, verbose = False)
            text = reader.readtext(img, detail = 0)
            text = self.text_preprocessing(text)
            preds.append(text)
        print('Done.')
        return preds

     -------------------------------------- 455.9/455.9 MB 1.1 MB/s eta 0:00:00
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached flatbuffers-22.9.24-py2.py3-none-any.whl (26 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached termcolor-2.0.1-py3-none-any.whl (5.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached wrapt-1.14.1-cp39-cp39-win_amd64.whl (35 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached h5py-3.7.0-cp39-cp39-win_amd64.whl (2.6 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl (895 kB)
  U

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\aeavy\\anaconda3\\envs\\py397\\Lib\\site-packages\\wrapt\\_wrappers.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
train_predicts = easyocr_model.prediction(train_df['img_path'].values)

In [ ]:
train_predicts

In [ ]:
train_predict_df = train_df.copy()
train_predict_df['text'] = train_predicts
print('Train Accuracy : ', get_accuracy(train_df, train_predict_df))

In [ ]:
test_predicts = easyocr_model.prediction(test_df['img_path'].values)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['text'] = test_predicts
submit.to_csv('./easyocr02.csv', index=False, encoding="utf-8-sig")